In [9]:
import pandas as pd
import numpy as np
import statsmodels.api as sm

In [10]:
df = pd.read_csv("../data/processed/merged_dataset.csv", index_col=0, parse_dates=True)
df.head()

,AAPL,ABBV,ABT,ADBE,AMZN,AVGO,AXP,BA,BAC,BLK,...,V,WFC,XOM,Mkt-RF,SMB,HML,RMW,CMA,RF,Market
Date,,,,,,,,,,,,,,,,,,,,,
2019-01-03,-0.099608,-0.032949,-0.047194,-0.039498,-0.025241,-0.088951,-0.019520,-0.039869,-0.016026,-0.029377,...,-0.036037,-0.007882,-0.015354,-0.0245,0.0048,0.0118,-0.0022,0.0091,0.0001,-0.0244
2019-01-04,0.042690,0.032217,0.028541,0.048632,0.050064,0.009828,0.045061,0.052042,0.041531,0.036616,...,0.043081,0.029633,0.036870,0.0355,0.0035,-0.0069,-0.0011,-0.0062,0.0001,0.0356
2019-01-07,-0.002226,0.014596,0.014975,0.013573,0.034353,0.020366,0.005428,0.003149,-0.000782,0.002782,...,0.018032,-0.006465,0.005200,0.0094,0.0087,-0.0075,-0.0074,-0.0045,0.0001,0.0095
2019-01-08,0.019063,0.004647,-0.012006,0.014918,0.016612,-0.008026,0.004889,0.037853,-0.001957,0.012726,...,0.005439,-0.002099,0.007271,0.0101,0.0044,-0.0063,0.0034,-0.0012,0.0001,0.0102
2019-01-09,0.016982,-0.032823,0.016691,0.011819,0.001714,0.043250,0.001825,0.009691,0.009800,0.005730,...,0.011769,0.005469,0.005275,0.0052,0.0050,0.0011,0.0009,-0.0020,0.0001,0.0053


In [11]:
factor_cols = ["Market", "SMB", "HML", "RMW", "CMA"]

In [12]:
def run_factor_regression(stock_returns, factors):
    X = sm.add_constant(factors)
    y = stock_returns
    
    model = sm.OLS(y, X, missing="drop").fit()
    return model

In [13]:
betas = {}
for stock in df.columns:
    if stock in factor_cols or stock == "RF":  
        continue
    
    model = run_factor_regression(df[stock], df[factor_cols])
    betas[stock] = model.params

In [14]:
beta_df = pd.DataFrame(betas).T
beta_df.head()

,const,Market,SMB,HML,RMW,CMA
AAPL,0.000380,1.254850,-0.113513,-0.622495,0.501722,0.574723
ABBV,0.000304,0.620088,-0.248533,0.054880,0.037740,0.427909
ABT,-0.000061,0.817467,-0.334839,-0.154956,-0.150708,0.431312
ADBE,-0.000204,1.194701,-0.291829,-0.463005,0.052420,-0.502099
AMZN,0.000162,0.986684,-0.289262,-0.353755,-0.000934,-0.945441


In [15]:
beta_df.to_csv("../data/processed/factor_betas.csv")
print("Saved factor betas to data/processed/factor_betas.csv")

Saved factor betas to data/processed/factor_betas.csv
